In [41]:
from typing import Dict
import tempfile
from pathlib import Path
import numpy as np
from mlflow.tracking import MlflowClient
import mlflow
from mlflow.artifacts import download_artifacts

FPS = 10
B, T = 5, 50

def download_artifact_npz(run_id, artifact_path) -> Dict[str, np.ndarray]:
    client = MlflowClient()
    with tempfile.TemporaryDirectory() as tmpdir:
        # path = download_artifacts(run_id=run_id, artifact_path=artifact_path)
        path = f'/home/user/data/Ahmad/Ahmad/pydreamer/mlruns/0/{run_id}/artifacts/{artifact_path}'
        with Path(path).open('rb') as f:
            data = np.load(f)
            return {k: data[k] for k in data.keys()}  # type: ignore

def encode_gif(frames, fps):
    # Copyright Danijar
    from subprocess import Popen, PIPE
    h, w, c = frames[0].shape
    pxfmt = {1: 'gray', 3: 'rgb24'}[c]
    cmd = ' '.join([
        'ffmpeg -y -f rawvideo -vcodec rawvideo',
        f'-r {fps:.02f} -s {w}x{h} -pix_fmt {pxfmt} -i - -filter_complex',
        '[0:v]split[x][z];[z]palettegen[y];[x]fifo[x];[x][y]paletteuse',
        f'-r {fps:.02f} -f gif -'])
    proc = Popen(cmd.split(' '), stdin=PIPE, stdout=PIPE, stderr=PIPE)
    for image in frames:
        proc.stdin.write(image.tobytes())  # type: ignore
    out, err = proc.communicate()
    if proc.returncode:
        raise IOError('\n'.join([' '.join(cmd), err.decode('utf8')]))
    del proc
    return out

def make_gif(env_name, run_id, step, fps=FPS):
    dest_path = f'figures/dream_{env_name}_{step}.gif'
    artifact = f'd2_wm_dream/{step}.npz'
    data = download_artifact_npz(run_id, artifact)
    img = data['image_pred']
    print(img.shape)
    img = img[:B, :T].reshape((-1, 64, 64, 3))
    gif = encode_gif(img, fps)
    with Path(dest_path).open('wb') as f:
        f.write(gif)

In [42]:
# Montezuma

make_gif('atari_pong', 'e9fc2df4b33741099f1335f3ac6974e5', '0080001')
# make_gif('atari_pong', '83e5def4975242ccbf16a3ca8f62a674', '0500001')

FileNotFoundError: [Errno 2] No such file or directory: '/home/user/data/Ahmad/Ahmad/pydreamer/mlruns/0/e9fc2df4b33741099f1335f3ac6974e5/artifacts/d2_wm_dream/d2_wm_dream/0080001.npz'